In [269]:
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

commodities = ['GC=F','SI=F', 'CL=F'] #gold, silver, Crude Oil
bonds = ['^FVX'] #5 Year US Government Bonds
us_stocks = ['AAPL', 'APX', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS'
         'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE', 'V']

[*********************100%***********************]  1 of 1 downloaded
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 296 entries, 2017-01-03 to 2017-12-29
Data columns (total 6 columns):
Open         296 non-null float64
High         296 non-null float64
Low          296 non-null float64
Close        296 non-null float64
Adj Close    296 non-null float64
Volume       296 non-null int64
dtypes: float64(5), int64(1)
memory usage: 16.2 KB


In [261]:
def download_stock_data(start_date, end_date, asset_list):
    for i, asset in enumerate(asset_list):
        if i == 0:
            data = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = data.rename(columns={"Adj Close": asset})
            
        else:
            new_asset = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = all_prices.join(new_asset.rename(columns={"Adj Close": asset}))
    
    all_prices.to_csv('./Data/Yahoo_latest_prices.csv')
    return

In [262]:
def process_stock_data(all_prices):
    n_time, n_assets = all_prices.shape
    #Calculate returns
    all_returns = all_prices.pct_change(1).apply(lambda a : a + 1)
    
    # Calculate relative price time series
    all_prices_norm = all_returns.copy()
    for i in range(n_time):
        if i ==0:
            all_prices_norm.iloc[0] = np.ones(n_assets)
        else:
            for j in range(n_assets):
                if math.isnan(all_prices_norm.iloc[i,j]):
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i-1,j]
                else:
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i,j] * all_prices_norm.iloc[i-1,j]
                    
                    
    return all_prices_norm, all_returns

In [270]:
# download stock data and safe as csv in /Data folder
s = download_stock_data('2000-01-01','2018-11-01', ['^GSPC', 'AAPL', 'APX', ])

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


FileNotFoundError: [Errno 2] No such file or directory: './Data/Yahoo_latest_prices.csv'

In [256]:
# read in the data again
all_prices = pd.read_csv('./Data/Yahoo_latest_prices.csv', header=0, index_col = 0)
all_prices.index = pd.to_datetime(data.index)

#get values we need
all_prices_norm, all_returns = process_stock_data(all_prices)

In [258]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf

# Print prices
iplot(all_prices.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Prices',title='Prices'))

# Print returns
iplot(all_returns.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Returns',title='Returns'))

# Start normalized prices
iplot(all_prices_norm.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Relative Price',title='Relative Prices'))